In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim

# Assumiamo che i tuoi dati siano già in formato numpy e normalizzati tra 0 e 1
# Carica i dati come tensori
images_tensor = torch.tensor(augmented_train_images_normalized, dtype=torch.float32)
labels_tensor = torch.tensor(augmented_train_labels, dtype=torch.long)

# Aggiungi una dimensione per il canale (1 canale per immagini in scala di grigio)
images_tensor = images_tensor.unsqueeze(1)  # Shape: (7750, 1, 150, 150)
images_tensor = images_tensor.squeeze(-1)
print(images_tensor.shape)

# Splitta i dati in 80% training e 20% validazione (stratificato)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(sss.split(images_tensor, labels_tensor))

train_images_tensor = images_tensor[train_idx]
val_images_tensor = images_tensor[val_idx]

train_labels_tensor = labels_tensor[train_idx]
val_labels_tensor = labels_tensor[val_idx]

# Crea i dataset e DataLoader
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_images_tensor, val_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modello e device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNeXtKAN().to(device)

# Definizione della funzione di perdita e ottimizzatore
criterion = nn.CrossEntropyLoss()  # Per classificazione multi-classe o binaria
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Informazioni sugli split
print(f"Train set: {len(train_dataset)} samples")
print(f"Validation set: {len(val_dataset)} samples")

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, patience=10):
    best_val_accuracy = 0.0
    best_val_loss = float("inf")
    epochs_without_improvement = 0  # Conta quante epoche senza miglioramento

    for epoch in range(num_epochs):
        model.train()  # Modalità allenamento
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Azzeramento dei gradienti
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calcolo della perdita
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Calcolo delle metriche
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels).item()
            total_preds += labels.size(0)

        # Calcolo della loss media e accuratezza
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_preds / total_preds

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")

        # Fase di validazione
        model.eval()  # Modalità valutazione
        val_loss = 0.0
        val_correct_preds = 0
        val_total_preds = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calcolo della perdita
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                # Calcolo delle metriche
                _, preds = torch.max(outputs, 1)
                val_correct_preds += torch.sum(preds == labels).item()
                val_total_preds += labels.size(0)

        # Calcolo della loss e accuratezza di validazione
        val_loss /= len(val_loader.dataset)
        val_accuracy = val_correct_preds / val_total_preds

        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        # Early Stopping: fermarsi se non c'è miglioramento
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_accuracy = val_accuracy
            epochs_without_improvement = 0
            # Salva il modello con la miglior loss di validazione
            torch.save(model.state_dict(), "best_model.pth")
            print("Model saved!")
        else:
            epochs_without_improvement += 1
            print(f"Early Stopping Counter: {epochs_without_improvement}/{patience}")

        # Se il numero di epoche senza miglioramenti supera la pazienza, fermati
        if epochs_without_improvement >= patience:
            print("Early stopping triggered. Stopping training.")
            break

# Avvia l'allenamento
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, patience=10)
